In [1]:
import pandas as pd


In [2]:
f = open("amazon/Home_and_Kitchen.json", "r")
f2 = open("amazon/Home_and_Kitchen_8.json", "w")

num_lines_before = 0
num_lines_after = 0
for line in f:
    num_lines_before += 1
    if hash(line) % 8 == 0:
        f2.write(line)
        num_lines_after += 1

f.close()
f2.close()

print(num_lines_before)
print(num_lines_after)

21928568
2740119


In [3]:
f = open("amazon/meta_Home_and_Kitchen.json", "r")

print(f.readline())

f.close()

{"category": ["Home & Kitchen", "Kitchen & Dining", "Dining & Entertaining", "Dinnerware", "Plates", "Dinner Plates"], "tech1": "", "description": ["It was a time honored tradition among the early American families that when someone deserved special praise or attention they were served dinner on the Red Plate. Observe the special occasions of each"], "fit": "", "title": "You Are Special Today Red Plate [With Red Pen]", "also_buy": ["B0001XR2F2", "B01LY51HUN", "B07CXZ9C5B", "0310258952"], "image": [], "tech2": "", "brand": "Waechtersbach USA", "feature": [], "rank": [">#39,665 in Kitchen & Dining (See Top 100 in Kitchen & Dining)", ">#93 in Kitchen & Dining > Tabletop > Dinnerware > Plates > Dinner Plates", ">#19,436 in Home & Kitchen > Kitchen & Dining > Food Service Equipment & Supplies"], "also_view": ["B0001XR2F2", "B00MOFKX1A", "B07G3LN13B", "B07CYXMFFH", "B01LY51HUN", "B06XCK743L", "B00PV4FHZI", "B07C1WTCL4", "B001ARPMD2", "B00847B0IC", "B000NCW2RE", "B06ZZ1GLQR"], "main_cat": "Am

In [4]:
import json

f = open("amazon/Home_and_Kitchen_8.json", "r")
f2 = open("amazon/Home_and_Kitchen_8_new.txt", "w")

# line = f.readline()
# line = json.loads(line)

# print(type(line['reviewText']))
num_lines = 0

for line in f:
    tmp = json.loads(line)
    try:
        if len(tmp['reviewText']) > 5:
            f2.write(tmp['reviewText'] + "\n")
            num_lines += 1
    except:
        continue

f.close()
f2.close()

print(num_lines)



2703639


In [5]:
from gensim.utils import tokenize


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

with open("amazon/Home_and_Kitchen_8_new.txt", "r") as f:
    tokenized_texts = [list(tokenize(text)) for text in f]
    
print(tokenized_texts[0][:10])
print(len(tokenized_texts))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Erik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['Great', 'product', 'love', 'it']
3588963


In [6]:

filtered = [[w for w in text if not w.lower() in stops] for text in tokenized_texts]

print(filtered[0][:10])

['Great', 'product', 'love']


In [7]:
from gensim.parsing.porter import PorterStemmer

stemmer = PorterStemmer()
stemmed_texts = [[stemmer.stem(word) for word in text] for text in filtered]

stemmed_texts[0][:10]

['great', 'product', 'love']

In [8]:
from gensim.corpora.dictionary import Dictionary


dictionary = Dictionary(stemmed_texts)
# A corpus is a sparse datastore containing the number of times each word appears in each document.
corpus = [dictionary.doc2bow(text) for text in stemmed_texts]



In [9]:
top_words_in_doc_0 = sorted(corpus[0], key=lambda e: e[1], reverse=True)[:10]
for word_index, count in top_words_in_doc_0:
  print(f'{dictionary[word_index]}   \tindex: {word_index}\tcount: {count:,}')

great   	index: 0	count: 1
love   	index: 1	count: 1
product   	index: 2	count: 1


In [10]:
print(len(stemmed_texts))
print(len(dictionary))
print(len(corpus))


3588963
138627
3588963


In [11]:
from gensim import models, similarities


model = models.LdaMulticore(corpus, id2word=dictionary, num_topics=10, workers=14)

In [12]:
print(model.show_topics(num_topics=10, num_words=10))

[(0, '0.084*"great" + 0.060*"product" + 0.038*"qualiti" + 0.037*"work" + 0.036*"good" + 0.033*"price" + 0.016*"well" + 0.014*"excel" + 0.014*"expect" + 0.014*"thank"'), (1, '0.030*"coffe" + 0.023*"cup" + 0.020*"us" + 0.014*"water" + 0.014*"make" + 0.013*"mug" + 0.013*"hot" + 0.011*"tea" + 0.011*"keep" + 0.011*"drink"'), (2, '0.026*"us" + 0.018*"sheet" + 0.017*"wash" + 0.015*"well" + 0.014*"pan" + 0.013*"cook" + 0.012*"like" + 0.009*"set" + 0.009*"on" + 0.008*"make"'), (3, '0.019*"on" + 0.016*"mattress" + 0.012*"like" + 0.011*"time" + 0.010*"purchas" + 0.010*"us" + 0.010*"review" + 0.010*"year" + 0.009*"bui" + 0.009*"bought"'), (4, '0.023*"us" + 0.017*"easi" + 0.016*"cut" + 0.014*"chair" + 0.011*"knife" + 0.011*"great" + 0.010*"would" + 0.009*"recommend" + 0.008*"good" + 0.008*"imag"'), (5, '0.027*"us" + 0.017*"water" + 0.017*"clean" + 0.013*"vacuum" + 0.011*"work" + 0.010*"rug" + 0.010*"on" + 0.010*"get" + 0.010*"like" + 0.009*"time"'), (6, '0.043*"love" + 0.020*"perfect" + 0.020*"us" 

In [13]:
import numpy as np

for ix in range(10):
  top10 = np.argsort(model.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')

0:  thank expect excel well price good work qualiti product great
1:  drink keep tea hot mug make water us cup coffe
2:  make on set like cook pan well wash sheet us
3:  bought bui year review us purchas time like mattress on
4:  imag good recommend would great knife chair cut easi us
5:  time like get on rug work vacuum clean water us
6:  easi small on bag great fit size us perfect love
7:  great qualiti pictur gift like beauti nice color love look
8:  time piec replac get us work back return would on
9:  smell look light us comfort like great room bed pillow


In [14]:
#trying without stemmed words

new_corpus = [dictionary.doc2bow(text) for text in filtered]

model = models.LdaMulticore(new_corpus, id2word=dictionary, num_topics=10, workers=14)

In [15]:
for ix in range(10):
  top10 = np.argsort(model.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')

0:  two box used would came get use mattress first time
1:  used good price job clean well easy works use great
2:  like use get mug cold lid hot cup coffee water
3:  bag really love great together like sturdy put easy nice
4:  gift purchase good review received happy would recommend love product
5:  use fit would big like perfect make little small size
6:  unit vacuum even return could buy get back like would
7:  rug daughter great set love beautiful color bed perfect room
8:  cute nice gift look like expected good made price well
9:  oven sheet blanket stick good use like pan soft pillow


In [16]:
from gensim.models.coherencemodel import CoherenceModel

cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)


umass:  -3.014806708032802
c_v:  0.5428692998435725


In [17]:
# try with less topics

model = models.LdaMulticore(new_corpus, id2word=dictionary, num_topics=5, workers=14)

In [18]:
for ix in range(5):
  top10 = np.argsort(model.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')

0:  like pan great received price used loved gift use product
1:  color would price well love size good product nice great
2:  mattress really together fit bed use like put easy well
3:  light hot back really get time expected would water like
4:  well would make great get cup time use coffee love


In [19]:
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)


umass:  -2.8200038535524636
c_v:  0.517487878292162


In [20]:
# adding more stop words

stops.add("like")
stops.add("love")
stops.add("loved")
stops.add("loves")
stops.add("use")
stops.add("get")
stops.add("used")
stops.add("would")
stops.add("even")

print(stops)


{'yourselves', 'than', 't', 'above', 'shouldn', "mightn't", "don't", 'yourself', 'these', "didn't", "you'll", 'do', 'were', 'wouldn', 'y', 'over', 'be', 'any', 'to', 'where', 'more', 'each', 'your', 'why', 'use', 'haven', 'those', 'out', 'yours', 'or', 'as', 'such', 'about', 'a', 'can', 'needn', 'ma', 'o', 'until', 'own', 'further', "wouldn't", 'get', 'aren', 're', 'this', "you'd", 'having', 'she', 'the', "shan't", "mustn't", 'by', 'now', "won't", 'themselves', 'off', 'mightn', 'their', 'itself', 'too', 'theirs', 'into', 'its', 'ourselves', 'hers', 'my', 'am', 'just', "should've", 'at', "couldn't", 'her', 'shan', 'doesn', 'and', 'himself', 'what', 'not', "wasn't", 'loved', 'ain', 'our', 'him', 'm', 'used', 'will', 'whom', 'before', 'so', 'that', 'which', 'for', "that'll", 'didn', 'was', 'if', 'most', 'has', 'being', 'after', 'in', 'on', 'here', "shouldn't", 'other', 'doing', 'with', 'i', "hadn't", 'he', 'there', 'don', 'did', 'like', 'myself', 'loves', "you've", 'then', 'hasn', 'does',

In [21]:
filtered = [[w for w in text if not w.lower() in stops] for text in tokenized_texts]

print(filtered[0][:10])

dictionary = Dictionary(filtered)
# A corpus is a sparse datastore containing the number of times each word appears in each document.
corpus = [dictionary.doc2bow(text) for text in filtered]

['Great', 'product']


In [22]:
model = models.LdaMulticore(corpus, id2word=dictionary, num_topics=5, workers=20)

In [23]:
for ix in range(5):
  top10 = np.argsort(model.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')
    

0:  hot cup pan water Works clean one easy great coffee
1:  really one made easy great perfect Perfect size well small
2:  bed good really little put room one nice great well
3:  buy gift well Great good price one product great quality
4:  work months back unit top first lid water time one


In [24]:
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)


umass:  -2.9400663032110743
c_v:  0.4880056985920172


In [25]:
# try with more topics

model = models.LdaMulticore(corpus, id2word=dictionary, num_topics=15, workers=14)

In [26]:
for ix in range(15):
  top10 = np.argsort(model.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')

0:  side put soft top fit little well nice one mattress
1:  sheets well good much price bought vacuum one pillow bed
2:  ordered return together put received box color back product one
3:  curtain one price shower small quality good perfect size Great
4:  old first make time ice bought ever best years one
5:  great easy make grinder one maker cups machine cup coffee
6:  warm pillows nice little air Perfect fan chair light room
7:  need easy nice kitchen much table space well great set
8:  product much buy blender Awesome fabric money well great one
9:  top cup mug one glass cold coffee lid hot water
10:  long battery still last first work well unit one time
11:  ref ie product class hook normal B link wine glasses
12:  much one oven good well Easy pan great clean easy
13:  buy Bought rice water year Christmas bought great one gift
14:  looks made nice Great price good well quality product great


In [27]:
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)

umass:  -3.0560961233570483
c_v:  0.5594208013130275


In [28]:
# try with 7 topics

model = models.LdaMulticore(corpus, id2word=dictionary, num_topics=7, workers=14)

In [29]:
for ix in range(7):
  top10 = np.argsort(model.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')

0:  sheets really room color pillow bed well great soft nice
1:  make time chair bought years made set well one mattress
2:  perfect well vacuum easy bag cute one Great gift great
3:  arrived looks item Great one great price good quality product
4:  unit heat ice back top hot time lid smell water
5:  hot years clock mug great bought time cup coffee one
6:  nice fit perfect small well little one put easy great


In [30]:
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)

umass:  -2.8411466628708757
c_v:  0.5220114469112925


In [31]:
model = models.LdaMulticore(corpus, id2word=dictionary, num_topics=6, workers=20)

for ix in range(6):
  top10 = np.argsort(model.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')

cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)    
    

0:  good easy pan first great review clean time product one
1:  pretty good pot set product much nice made well one
2:  easy nice works good price well product quality Great great
3:  bag great fit good top well size little small easy
4:  cold great ice make time hot cup one water coffee
5:  pillow back really color nice soft room bed bought one
umass:  -2.8641187015656944
c_v:  0.5053322605290452


In [32]:
model = models.LdaMulticore(corpus, id2word=dictionary, num_topics=4, workers=20)

for ix in range(4):
  top10 = np.argsort(model.get_topics()[ix])[-10:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top10])}')

cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)  

0:  room quality time well price bed good product one great
1:  top look looks really little great put well one nice
2:  make much time hot cup one great well water coffee
3:  made gift well good easy product quality great one Great
umass:  -2.7389961461909933
c_v:  0.43561696978755293


In [33]:
# time to try with poor ratings only
import json


f = open("amazon/Home_and_Kitchen.json", "r")
f2 = open("amazon/Home_and_Kitchen_poor.txt", "w")

num_lines_before = 0
num_lines_after = 0
for line in f:
    if num_lines_after >= 2_000_000:
        break
    num_lines_before += 1
    tmp = json.loads(line)
    try:
        if len(tmp['reviewText']) > 5 and tmp['overall'] == 0.0 or tmp['overall'] == 1.0 or tmp['overall'] == 2.0:
            f2.write(tmp['reviewText'] + "\n")
            num_lines_after += 1
    except:
        continue

f.close()
f2.close()

print(num_lines_before)
print(num_lines_after)

18125643
2500000


In [34]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

print(stops)

{'yourselves', 'than', 't', 'above', 'shouldn', "mightn't", "don't", 'yourself', 'these', "didn't", "you'll", 'do', 'were', 'wouldn', 'y', 'over', 'be', 'any', 'to', 'where', 'more', 'each', 'your', 'why', 'haven', 'those', 'out', 'yours', 'or', 'as', 'such', 'about', 'a', 'can', 'needn', 'ma', 'o', 'until', 'own', 'further', "wouldn't", 'aren', 're', 'this', "you'd", 'having', 'she', 'the', "shan't", "mustn't", 'by', 'now', "won't", 'themselves', 'off', 'mightn', 'their', 'itself', 'too', 'theirs', 'into', 'its', 'ourselves', 'hers', 'my', 'am', 'just', "should've", 'at', "couldn't", 'her', 'shan', 'doesn', 'and', 'himself', 'what', 'not', "wasn't", 'ain', 'our', 'him', 'm', 'will', 'whom', 'before', 'so', 'that', 'which', 'for', "that'll", 'didn', 'was', 'if', 'most', 'has', 'being', 'after', 'in', 'on', 'here', "shouldn't", 'other', 'doing', 'with', 'i', "hadn't", 'he', 'there', 'don', 'did', 'myself', "you've", 'then', 'hasn', 'does', 'are', 'you', 'of', 'who', 'herself', 'very', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Erik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
with open("amazon/Home_and_Kitchen_poor.txt", "r") as f:
    tokenized_texts = [list(tokenize(text)) for text in f]
    
print(tokenized_texts[0][:10])
print(len(tokenized_texts))

['I', 'truly', 'wanted', 'to', 'like', 'this', 'item', 'It', 'fell', 'apart']
3673242


In [36]:
filtered = [[w for w in text if not w.lower() in stops] for text in tokenized_texts]

dictionary = Dictionary(filtered)

print(filtered[0][:10])

['truly', 'wanted', 'like', 'item', 'fell', 'apart', 'installing', 'screwdriver']


In [37]:
new_corpus = [dictionary.doc2bow(text) for text in filtered]

model = models.LdaMulticore(new_corpus, id2word=dictionary, num_topics=5, workers=20)

In [39]:
for ix in range(5):
  top8 = np.argsort(model.get_topics()[ix])[-8:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top8])}')

cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence)  

0:  use would machine hot one cup coffee water
1:  would use item one product pan made quality
2:  handle pillows one first money use product broke
3:  box plastic fit together put product one would
4:  plastic pillow would get one like lid use
umass:  -2.8289453797083977
c_v:  0.5683332336678729


In [40]:
stops.add('one')
stops.add('would')

filtered = [[w for w in text if not w.lower() in stops] for text in tokenized_texts]

dictionary = Dictionary(filtered)

print(filtered[0][:10])

['truly', 'wanted', 'like', 'item', 'fell', 'apart', 'installing', 'screwdriver']


In [41]:
new_corpus = [dictionary.doc2bow(text) for text in filtered]

model = models.LdaMulticore(new_corpus, id2word=dictionary, num_topics=5, workers=20)

In [42]:
for ix in range(5):
  top8 = np.argsort(model.get_topics()[ix])[-8:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top8])}')

cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence) 

0:  get bought product working months coffee time work
1:  plastic much fit money small use made product
2:  first good get used like time water use
3:  use vacuum back get item Amazon return product
4:  product even cheap put get back quality like
umass:  -2.5976535452789697
c_v:  0.5218575408496833


In [43]:
model = models.LdaMulticore(new_corpus, id2word=dictionary, num_topics=4, workers=20)

for ix in range(4):
  top8 = np.argsort(model.get_topics()[ix])[-8:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top8])}')

cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence) 

0:  money back product months time use get water
1:  back put get plastic like product quality use
2:  color get bought pillow bed even product like
3:  money get use made time small work product
umass:  -2.5978808075371562
c_v:  0.4718693370593061


In [ ]:
model = models.LdaMulticore(new_corpus, id2word=dictionary, num_topics=6, workers=20)

for ix in range(6):
  top8 = np.argsort(model.get_topics()[ix])[-8:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top8])}')

cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence) 


import pickle

with open("topic.model", "wb") as f:
    pickle.dump(model, f)


0:  time Amazon mattress get money product return back
1:  money used pillow get time like use water
2:  color made picture disappointed cheap product quality like
3:  even reviews pan hot cup product get coffee
4:  worked clock stopped use months time working work
5:  hold two broken first product plastic broke use


In [46]:
model = models.LdaMulticore(new_corpus, id2word=dictionary, num_topics=7, workers=20)

for ix in range(7):
  top8 = np.argsort(model.get_topics()[ix])[-8:]
  print(f'{ix}:  {" ".join([dictionary[index] for index in top8])}')

cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='u_mass')
coherence = cm.get_coherence()
print("umass: ", coherence)
cm = CoherenceModel(model=model, texts=filtered, dictionary=dictionary, coherence='c_v')
coherence = cm.get_coherence()
print("c_v: ", coherence) 

0:  clean like air much small use vacuum mattress
1:  make get like time used product pan use
2:  box received item disappointed broken use quality product
3:  fit quality time even put product get like
4:  new get back product Amazon months working return
5:  get use money like cheap made broke plastic
6:  hot like use back work time water coffee
umass:  -2.627010918628895
c_v:  0.5202661200062719
